In [1]:
# Code has been changed to incorporate changes for Python 3
# All the imports
import numpy as np
import urllib
import bs4 # This is beautiful soup
import time
import operator
import socket
import _pickle
import re # Regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

In [2]:
# Column names to be read from CSV
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.user',
    sep = '|', names = u_cols)
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [3]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.data',
    sep = '\t', names = r_cols)
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
# The movie file contains colums indicating the movie's gneres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date',
          'imdb_url']
movies = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.item',
     sep = '|', names = m_cols, usecols = range(5))
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [8]:
print(movies.dtypes)
print(movies.describe())
# Here movies.describe() only generate statistics summary for int64
# and float64 data types by default it includes only numeric data types

movie_id                int64
title                  object
release_date           object
video_release_date    float64
imdb_url               object
dtype: object
          movie_id  video_release_date
count  1682.000000                 0.0
mean    841.500000                 NaN
std     485.695893                 NaN
min       1.000000                 NaN
25%     421.250000                 NaN
50%     841.500000                 NaN
75%    1261.750000                 NaN
max    1682.000000                 NaN


In [11]:
users.head()
print(users['occupation'].head()) # Prints only occupation column
columns_you_want = ['occupation', 'sex']
print(users[columns_you_want].head()) # Prints only occupation and sex
print(users.head()) # Prints first five rows
print(users.iloc(3)) 

0    technician
1         other
2        writer
3    technician
4         other
Name: occupation, dtype: object
   occupation sex
0  technician   M
1       other   F
2      writer   M
3  technician   M
4       other   F
   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213


In [16]:
# Selecting users older than 25
print("Users older than 25")
oldUsers = users[users.age > 25]
oldUsers.head()

Users older than 25


,user_id,age,sex,occupation,zip_code
1,2,53,F,other,94043
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201


In [15]:
# Selecting users aged 40 and male
print("\n Users aged 40 and male")
fortyAndMale = users[(users.age == 40) & (users.sex == 'M')]
fortyAndMale.head(3)


 Users aged 40 and male


,user_id,age,sex,occupation,zip_code
18,19,40,M,librarian,02138
82,83,40,M,other,44133
115,116,40,M,healthcare,97232


In [19]:
# Users who are female and programmers
selected_users = users[(users.sex == 'F') &
                        (users.occupation == 'programmer')]
print(selected_users.describe()) # Statistics summary

# Two ways to operate on columns
print(selected_users.age.mean())
print(selected_users['age'].mean())

          user_id        age
count    6.000000   6.000000
mean   411.166667  32.166667
std    149.987222   5.115336
min    292.000000  26.000000
25%    313.000000  28.250000
50%    378.000000  32.000000
75%    416.750000  36.500000
max    698.000000  38.000000
32.1666666667
32.1666666667


In [27]:
print(ratings.head())
# Split data
grouped_data = ratings['movie_id'].groupby(ratings['user_id'])
# Get no of ratings per user
ratings_per_user = grouped_data.count()
print(ratings_per_user.head(5))

   user_id  movie_id  rating  unix_timestamp
0      196       242       3       881250949
1      186       302       3       891717742
2       22       377       1       878887116
3      244        51       2       880606923
4      166       346       1       886397596
user_id
1    272
2     62
3     54
4     24
5    175
Name: movie_id, dtype: int64


In [36]:
# Get Average rating per movie
grouped_data = ratings['rating'].groupby(ratings['movie_id'])
# Average and combine
average_ratings = grouped_data.mean()
print(average_ratings.head())

movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
Name: rating, dtype: float64


In [34]:
# Print good movies i.e. movies with highest rating
maximum_rating = average_ratings.max()
good_movies_ids = average_ratings[average_ratings == maximum_rating].index
print("Good movie ids:")
print(good_movies_ids)

print("Best Movie Titles")
print(movies[movies['movie_id'].isin(good_movies_ids)].title) 
# isin is a Dataframe function from pandas

Good movie ids:
Int64Index([814, 1122, 1189, 1201, 1293, 1467, 1500, 1536, 1599, 1653], dtype='int64', name='movie_id')
Best Movie Titles
813                         Great Day in Harlem, A (1994)
1121                       They Made Me a Criminal (1939)
1188                                   Prefontaine (1997)
1200           Marlene Dietrich: Shadow and Light (1996) 
1292                                      Star Kid (1997)
1466                 Saint of Fort Washington, The (1993)
1499                            Santa with Muscles (1996)
1535                                 Aiqing wansui (1994)
1598                        Someone Else's America (1995)
1652    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object


In [41]:
# No of ratings per movie
how_many_ratings = grouped_data.count()
# Maximum rating
print("Maximum rating = ", maximum_rating)
print("Numbers of ratings per movie where avg rating is equal \
to maximum rating")
print(how_many_ratings[average_ratings == maximum_rating])      

Maximum rating =  5.0
Numbers of ratings per movie where avg rating is equal to maximum rating
movie_id
814     1
1122    1
1189    3
1201    1
1293    3
1467    2
1500    2
1536    1
1599    1
1653    1
Name: rating, dtype: int64


In [42]:
# Passing a function
average_ratings = grouped_data.apply(lambda f: f.mean())
average_ratings.head()

movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
Name: rating, dtype: float64

In [43]:
# Average rating per user
grouped_data = ratings['rating'].groupby(ratings['user_id'])
average_ratings = grouped_data.mean()
average_ratings.head()

user_id
1    3.610294
2    3.709677
3    2.796296
4    4.333333
5    2.874286
Name: rating, dtype: float64

In [44]:
# Find if occupation is male or female dominant
grouped_data = users['sex'].groupby(users['occupation'])
male_dominant_occupations = grouped_data.apply(lambda f: 
                                         sum(f == 'M') > sum(f == 'F'))
print(male_dominant_occupations)
print('\n')

occupation
administrator     True
artist            True
doctor            True
educator          True
engineer          True
entertainment     True
executive         True
healthcare       False
homemaker        False
lawyer            True
librarian        False
marketing         True
none              True
other             True
programmer        True
retired           True
salesman          True
scientist         True
student           True
technician        True
writer            True
Name: sex, dtype: bool




In [45]:
print('Number of male users: ')
print(sum(users['sex'] == 'M'))
print('Number of female users:')
print(sum(users['sex'] == 'F'))

Number of male users: 
670
Number of female users:
273


In [47]:
# Seeing html of homepage of beautifulsoup
url = 'http://www.crummy.com/software/BeautifulSoup'
source = urllib.request.urlopen(url).read()
print(source)

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN"\n"http://www.w3.org/TR/REC-html40/transitional.dtd">\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<title>Beautiful Soup: We called him Tortoise because he taught us.</title>\n<link rev="made" href="mailto:leonardr@segfault.org">\n<link rel="stylesheet" type="text/css" href="/nb/themes/Default/nb.css">\n<meta name="Description" content="Beautiful Soup: a library designed for screen-scraping HTML and XML.">\n<meta name="generator" content="Markov Approximation 1.4 (module: leonardr)">\n<meta name="author" content="Leonard Richardson">\n</head>\n<body bgcolor="white" text="black" link="blue" vlink="660066" alink="red">\n<img align="right" src="10.1.jpg" width="250"><br />\n\n<p>You didn\'t write that awful page. You\'re just trying to get some\ndata out of it. Beautiful Soup is here to help. Since 2004, it\'s been\nsaving programmers hours or days of work on quick-turnaround\nscreen sc

In [53]:
# Check for 'Alice' in source 
# Notice that source is bytes-like object
# So 'Alice', 'Soup', 'alien video games' need to be prefixed with b
print(b'Alice' in source)

# Count occurences of 'Soup'
print(source.count(b'Soup'))

# Find index of 'alien video games'
position = source.find(b'alien video games')
print(position)

# Checking if aliend video games exists at position
print(source[position: position + 20])

False
42
1639
